# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Direct Preference Optimization

__Мягкий дедлайн 16.11.25 23:59__ \
__Жесткий дедлайн 19.11.25 23:59__

### О задании

В этом задании вам предстоит обучить большую LLM для ответов на вопросы с помощью DPO, а также реализовать LoRA для эффективного обучения.

### Оценивание и штрафы

Максимально допустимая оценка за работу — __11 баллов__.

Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Мы настаиваем на том, чтобы вы оформили весь код в виде полноценного проекта. Этот ноутбук нужно рассматривать скорее как файл с условием, чем как место для написания массивного кода. За сдачу больших ноутбуков с кодом оценка будет снижена. Ответы на все вопросы в заданиях можно (нужно) писать в отчете.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

### План решения

<img src="https://miro.medium.com/v2/resize:fit:1400/1*lK6iJMz5CGh2fo7TsDn15A.png" alt="drawing" width="700"/>

Обучение следованию инструкциям с помощью DPO разбивается на два этапа:    
1. __Supervised Fine-tuning (SFT)__ – обучение базовой модели ответам на запросы в нужном формате.
2. __Direct Preference Optimization (DPO)__ – обучение SFT модели приоритизации "хороших" ответов.

Мы не хотим обучать модели целиком по двум причинам: 1) используемые модели очень большие; 2) нам требуется лишь выравнить модель с нашими предпочтениями, не внося в нее новых знаний, что не требует серьезного обучения. Поэтому мы будем использовать PEFT, а именно LoRA для обучения.

Таким образом, вам надо будет:
1. Реализовать и протестировать LoRA
2. Разобраться с данными и привести их к нужному формату
3. Обучить SFT модель
4. Обучить DPO модель
5. Порадоваться, что вы молодцы и со всем справились
6. (Опционально) сделать веб-интерфейс для вашей модели, переиспользуя код из первой домашки (мы можем выдать бонусы, если получится классно).

### О датасете

Мы будем работать с датасетом [Anthropic Helpful-Harmless](https://huggingface.co/datasets/Anthropic/hh-rlhf) для RLHF. В нем содержится 160к примеров ответов на вопросы с историей.

### Low-Rank Adaptation (LoRA)

<img src="https://heidloff.net/assets/img/2023/08/lora.png" alt="drawing" width="600"/>

__Задание 1 (3 балла).__ Реализуйте самостоятельно модуль LoRA для эффективного обучения LLM по схеме, описанной в [статье](https://arxiv.org/pdf/2106.09685). Встройте его в свою любимую LLM и убедитесь, что ошибка убывает при обучении параметров LoRA на безусловную генерацию. Для этого возьмите любые данные на свой выбор. Замерьте насколько уменьшилось число обучаемых параметров, как изменилась скорость во время forward и backward процессов и как изменились затраты по памяти. Сделайте выводы и напишите о них в отчете.

In [1]:
# from utils import get_tokenizer, get_dataloaders, count_trinable_params
# from lora import get_base_model, get_lora_model, train_model_exp, eval_model_exp
# from sft import get_sft_dataloaders, train_model_sft, eval_model_sft

In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import os

def get_tokenizer():
    os.environ['TOKENIZERS_PARALLELISM'] = 'false'

    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b")
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    return tokenizer

def get_dataloaders(dataset_name='Anthropic/hh-rlhf', train_len=5000, val_len=500):
    batch_size = 32
    train_dataset = load_dataset(dataset_name, data_dir="harmless-base", split=f'train[:{train_len}]')
    val_dataset = load_dataset(dataset_name, data_dir="harmless-base", split=f'train[{train_len}:{train_len + val_len}]')

    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=8)
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size, num_workers=8)

    return train_loader, val_loader

def count_trinable_params(model):
    return sum([p.numel() for p in model.parameters() if p.requires_grad])


import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM
import numpy as np
import time


class LoRALayer(nn.Module):
    def __init__(self, base_layer: nn.Module, lora_rank: int = 4) -> None:
        super(LoRALayer, self).__init__()
        self.lora_rank = lora_rank

        self.base_layer = base_layer
        self.base_layer.eval()
        for param in base_layer.parameters():
            param.requires_grad = False

        self.lora_A = nn.Linear(base_layer.in_features, lora_rank, bias=False)
        self.lora_B = nn.Linear(lora_rank, base_layer.out_features, bias=False)

        nn.init.normal_(self.lora_A.weight)
        nn.init.zeros_(self.lora_B.weight)


class LoRALinear(LoRALayer):
    def __init__(self, base_layer: nn.Linear, lora_rank: int = 4) -> None:
        super().__init__(base_layer, lora_rank)

    def forward(self, x: Tensor) -> Tensor:
        base_out = self.base_layer(x)

        if self.lora_rank > 0:
            return base_out + self.lora_B(self.lora_A(x))

        return base_out


def get_base_model():
    model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1.4b")
    return model

def substitute_layers(root, lora_rank):
    for name, module in root.named_children():
        if isinstance(module, nn.Linear):
            setattr(root, name, LoRALinear(module, lora_rank))
        elif len(list(module.children())) > 0:
            substitute_layers(module, lora_rank)

def apply_lora_to_gpt2(model: nn.Module, lora_rank: int):
    for name, param in model.named_parameters():
        param.requires_grad = False
    substitute_layers(model, lora_rank)
    return model

def get_lora_model(lora_rank: int = 4):
    base_model = get_base_model()
    with_lora_model = apply_lora_to_gpt2(base_model, lora_rank)

    return with_lora_model

def train_model_exp(model, tokenizer, train_loader, val_loader, n_epochs=1, max_length=512):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    opt = torch.optim.Adam(model.parameters(), lr=2e-4)
    model.to(device)

    torch.cuda.reset_max_memory_allocated()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()
    start_time = time.perf_counter()

    for epoch in range(n_epochs):
        print(f'epoch {epoch + 1}/{n_epochs}')
        model.train()
        losses = []
        n_steps = len(train_loader)
        for i, batch in enumerate(train_loader):
            texts = batch['chosen']
            inputs = tokenizer(texts, max_length=max_length, truncation=True,
                               padding='max_length', return_tensors='pt').to(device)
            outputs = model(**inputs)
            next_word_logits = outputs.logits[:, : -1, :]
            true_next_tokens = inputs['input_ids'][:, 1:]
            loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
            losses.append(loss.item())

            loss.backward()
            opt.step()
            opt.zero_grad()
            print(f'iter {i+1}/{n_steps};  train loss: {loss.item()}')

        print('Epoch train loss:', np.mean(losses))
        print()

        model.eval()
        eval_model_exp(model, tokenizer, val_loader, max_length=max_length)

    torch.cuda.synchronize()
    time_elapsed = time.perf_counter() - start_time
    memory = torch.cuda.max_memory_allocated()

    print(f'elapsed time: {round(time_elapsed, 2)} s')
    print(f'memory: {memory}')

    return model

@torch.no_grad()
def eval_model_exp(model, tokenizer, val_loader, max_length=512):
    device = model.device
    model.eval()
    losses = []

    for batch in val_loader:
        texts = batch['chosen']
        inputs = tokenizer(texts, max_length=max_length, truncation=True,
                           padding='max_length', return_tensors='pt').to(device)
        outputs = model(**inputs)
        next_word_logits = outputs.logits[:, : -1, :]
        true_next_tokens = inputs['input_ids'][:, 1:]
        loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
        losses.append(loss.item())

    print("Val loss:", np.mean(losses))



In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Model, GPT2Tokenizer, RobertaPreTrainedModel, AutoModelForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader
import time
import psutil
import os
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [4]:
tokenizer = get_tokenizer()
train_loader, val_loader = get_dataloaders()

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

harmless-base/test.jsonl.gz:   0%|          | 0.00/743k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
base_model = get_base_model()
print(f'Обучаемых параметров: {count_trinable_params(base_model)}')
del base_model

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

Обучаемых параметров: 1414647808


In [7]:
lora_model = get_lora_model(lora_rank=4)
print(f'Обучаемых параметров с LoRA: {count_trinable_params(lora_model)}')

Обучаемых параметров с LoRA: 3355136


> В базовой модели 1.4 миллиарда параметров, в LoRA части - 3.3 миллиона

In [8]:
trained_lora = train_model_exp(lora_model, tokenizer, train_loader, val_loader, n_epochs=1, max_length=64)

epoch 1/1
iter 1/157;  train loss: 3.8980374336242676
iter 2/157;  train loss: 3.70688796043396
iter 3/157;  train loss: 2.9786553382873535
iter 4/157;  train loss: 2.5193307399749756
iter 5/157;  train loss: 2.2799265384674072
iter 6/157;  train loss: 2.1610307693481445
iter 7/157;  train loss: 2.0277392864227295
iter 8/157;  train loss: 2.1639516353607178
iter 9/157;  train loss: 1.9594208002090454
iter 10/157;  train loss: 1.8597580194473267
iter 11/157;  train loss: 1.8944169282913208
iter 12/157;  train loss: 1.924318790435791
iter 13/157;  train loss: 1.859330654144287
iter 14/157;  train loss: 2.0355265140533447
iter 15/157;  train loss: 1.9245599508285522
iter 16/157;  train loss: 1.8454298973083496
iter 17/157;  train loss: 1.7870506048202515
iter 18/157;  train loss: 1.676590919494629
iter 19/157;  train loss: 1.8981454372406006
iter 20/157;  train loss: 1.8736953735351562
iter 21/157;  train loss: 1.7892247438430786
iter 22/157;  train loss: 1.8236764669418335
iter 23/157;  

In [9]:
del lora_model

In [10]:
del trained_lora

### Supervised Fine-tuning

__Задание 2 (3 балла).__ Разбейте все примеры с "хорошими" ответами на запросы (все что идет до последнего "Assistant:") и ответы (все, начиная с последнего "Assistant:"). Дообучите модель [`pythia-1.4b`](https://huggingface.co/EleutherAI/pythia-1.4b) генерировать правильные ответы с помощью вашей LoRA. Одной эпохи вполне должно хватить для сходимости. Проверьте на нескольких случайных тестовых примерах, что модель ведет себя так, как надо.

In [12]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
import numpy as np
from numpy.random import randint
import time
import os


class SFTDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

def prepare_sft_data(dataset):
    processed_data = []

    for example in dataset:
        text = example['chosen']
        last_assistant_idx = text.rfind("Assistant:")

        if last_assistant_idx == -1:
            continue

        prompt = text[:last_assistant_idx].strip()
        response = text[last_assistant_idx:].strip()

        if len(prompt) and len(response):
            processed_data.append({
                'prompt': prompt,
                'response': response
            })

    return processed_data

def get_sft_dataloaders(train_loader, val_loader, batch_size=64):
    pocessed_train = prepare_sft_data(train_loader.dataset)
    processed_tval = prepare_sft_data(val_loader.dataset)

    pocessed_train_loader = DataLoader(SFTDataset(pocessed_train),
                                       shuffle=True, batch_size=batch_size, num_workers=8)
    pocessed_val_loader = DataLoader(SFTDataset(processed_tval),
                                     shuffle=False, batch_size=batch_size, num_workers=8)

    return pocessed_train_loader, pocessed_val_loader


@torch.no_grad()
def eval_model_sft(model, tokenizer, val_loader, max_length=512):
    device = model.device
    model.eval()
    losses = []

    for batch in val_loader:
        prompt = batch['prompt']
        response = batch['response']

        resp_inputs = tokenizer(response,max_length=max_length, truncation=True,
                           padding='max_length', return_tensors='pt')['input_ids'].to(device)

        len_test_ids = len(tokenizer(prompt, max_length=max_length, truncation=True,
                           padding='max_length', return_tensors='pt')['input_ids'].to(device))
        num_prompts = len(resp_inputs)

        space_for_prompts = torch.full([len_test_ids, num_prompts], fill_value=tokenizer.pad_token_id,
                                       dtype=torch.int64, device=device)
        batch['input_ids'] = torch.cat([space_for_prompts, resp_inputs['input_ids']], dim=1)
        batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), resp_inputs['attention_mask']], dim=1)

        outputs = model(**batch)
        next_word_logits = outputs.logits[:, num_prompts : -1, :]
        true_next_tokens = resp_inputs['input_ids'][:, num_prompts + 1:]
        loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
        losses.append(loss.item())

    print("Val loss:", np.mean(losses))


def train_model_sft(model, tokenizer, train_loader, val_loader, n_epochs=5, max_length=512):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    opt = torch.optim.Adam(model.parameters(), lr=2e-4)
    model.to(device)

    torch.cuda.reset_max_memory_allocated()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()
    start_time = time.perf_counter()

    for epoch in range(n_epochs):
        model.train()
        losses = []
        for batch in train_loader:
            prompt = batch['prompt']
            response = batch['response']

            resp_inputs = tokenizer(response, max_length=max_length, truncation=True,
                           padding='max_length', return_tensors='pt')['input_ids'].to(device)

            len_test_ids = len(tokenizer(prompt, max_length=max_length, truncation=True,
                           padding='max_length', return_tensors='pt')['input_ids'].to(device))
            num_prompts = len(resp_inputs)

            space_for_prompts = torch.full([len_test_ids, num_prompts], fill_value=tokenizer.pad_token_id,
                                        dtype=torch.int64, device=device)
            batch['input_ids'] = torch.cat([space_for_prompts, resp_inputs['input_ids']], dim=1)
            batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), resp_inputs['attention_mask']], dim=1)

            outputs = model(**batch)
            next_word_logits = outputs.logits[:, num_prompts : -1, :]
            true_next_tokens = resp_inputs['input_ids'][:, num_prompts + 1:]
            loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
            losses.append(loss.item())

            loss.backward()
            opt.step()
            opt.zero_grad()

        print('Train loss:', np.mean(losses))

        model.eval()
        eval_model_sft(model, tokenizer, val_loader, max_length=max_length)

    torch.cuda.synchronize()
    time_elapsed = time.perf_counter() - start_time
    memory = torch.cuda.max_memory_allocated()

    return model, time_elapsed, memory


def model_apply(model, tokenizer, prompt, max_length=32):
    model.eval()
    device = model.device
    n = 16

    test_input_ids = tokenizer(prompt, max_length=max_length, truncation=True,
                           padding='max_length', return_tensors='pt')['input_ids'].to(device)
    space_for_prompts = torch.full([len(test_input_ids), n], fill_value=tokenizer.pad_token_id,
                                dtype=torch.int64, device=device)

    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
    batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)

    length = randint(min(5, max_length), max_length + 1)
    for i in range(length):
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

    print("\nOutput:", tokenizer.decode(batch['input_ids'][0, n:].cpu().numpy().tolist()))



In [6]:
# your code here
sft_train_loader, sft_val_loader = get_sft_dataloaders(train_loader, val_loader)

In [ ]:
lora_model = get_lora_model(lora_rank=4)
trained_lora = train_model_sft(lora_model, tokenizer, sft_train_loader, sft_val_loader, n_epochs=1, max_length=64)

In [ ]:
del lora_model

In [ ]:
batch = sft_train_loader.dataset[0]
print('Prompt:')
print(batch['prompt'])
print('_' * 50)
print('True response:')
print(batch['response'])
print('_' * 50)

print('Predicted response:')
inputs = tokenizer(batch['prompt'], max_length=32, truncation=True,
                   padding='max_length', return_tensors='pt')
outputs = trained_lora.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
print('_' * 50)

### Direct Preference Optimization

__Задание 3 (3 балла).__ Реализуйте DPO согласно [статье](https://arxiv.org/pdf/2305.18290) и дообучите SFT модель с предыдущего шага. Одной эпохи так же должно хватить, но можно обучать и дольше. Убедитесь, что модель начинает отдавать предпочтение хорошим ответам. Проведите анализ. Стали ли ответы лучше, чем у SFT модели? Всегда ли модель отвечает хорошо или иногда плохо? Насколько легко модель ломается при изменении промптов?